In [9]:
#loading Librraries
from datasets import load_dataset
from rouge_score import rouge_scorer
import csv
import time
import gc
import getpass
import os
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

c:\Users\Manish\.conda\envs\capstone\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Manish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
#loading dataset
# Load the dataset
multi_lexsum = load_dataset("allenai/multi_lexsum", name="v20230518")
instances_list = list(multi_lexsum["validation"])


In [11]:
#OpenaAi api key removed for security. You can add your own or contact mp420@uowmail.edu.au [Manish Paikara 7640432] for the one used in the project
llm = OpenAI(openai_api_key="*")

c:\Users\Manish\.conda\envs\capstone\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [12]:
def get_full_text(index):
    entry = multi_lexsum['validation'][index]
    document_text = entry['sources']
    document_string = "".join(document_text)
    return document_string



In [13]:
def get_summary_long(index):
    entry = multi_lexsum['validation'][index]
    summary_text = entry['summary/long']
    return summary_text

In [14]:
def get_Case_ID(index):
    entry = multi_lexsum['validation'][index]
    case_id = entry['id']
    return case_id

In [15]:
get_Case_ID(5)

'JI-OH-0011'

In [3]:
def get_summary_openai(document_string):
    #
    # Optimize the chunk size and overlap
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=3500, chunk_overlap=500, length_function=len)
    text_chunks = text_splitter.split_text(document_string)
    print(len(text_chunks))

    # Create a list of LangChain documents from the chunks
    documents = [Document(page_content=chunk) for chunk in text_chunks]

    # Concise map prompt template
    map_template = "Summarize these {documents}, focusing on key legal arguments, evidence, claims, and judgments described in each segment."
    map_prompt = PromptTemplate.from_template(map_template)
    map_chain = LLMChain(llm=llm, prompt=map_prompt)

    # Concise reduce prompt template
    reduce_template = """Using the {doc_summaries} generate a summary that describes the entire document from a legal viewpoint"""
    reduce_prompt = PromptTemplate.from_template(reduce_template)
    reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

    combine_documents_chain = StuffDocumentsChain(
        llm_chain=reduce_chain, document_variable_name="doc_summaries"
    )
    reduce_documents_chain = ReduceDocumentsChain(
        combine_documents_chain=combine_documents_chain,
        collapse_documents_chain=combine_documents_chain,
        token_max=4000,
    )

    map_reduce_chain = MapReduceDocumentsChain(
        llm_chain=map_chain,
        reduce_documents_chain=reduce_documents_chain,
        document_variable_name="documents",
        return_intermediate_steps=False,
    )

    def summarize_pdf(documents):
        return map_reduce_chain.run(documents)

    result_summary = summarize_pdf(documents)
    return result_summary

# print(get_summary_openai(document_string))



In [1]:
#Trial code to check functionality
# summary_openai = get_summary_openai(document_string)
# print(summary_openai)

In [9]:
# entry = multi_lexsum['validation'][17]
# case_id = entry["id"]
# case_id

'PA-CA-0002'

In [22]:
def process_instances(iteration, csv_file_path='rogue_score_openai.csv'):
    refrence_summary = get_summary_long(iteration)
    generated_summary = get_summary_openai(str(get_full_text(iteration)))

    # print(refrence_summary)
    # print(generated_summary)

    # Initialize ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    score = scorer.score(generated_summary, refrence_summary )

    # print(score['rouge1'].precision)
    # print(score['rouge1'].recall)
    # print(score['rouge1'].fmeasure)
    # print(score['rouge2'].precision)
    # print(score['rouge2'].recall)
    # print(score['rouge2'].fmeasure)
    # print(score['rougeL'].precision)
    # print(score['rougeL'].recall)
    # print(score['rougeL'].fmeasure)

    with open(csv_file_path, 'a', newline='') as csvfile:
        fieldnames = [
            'Case_ID', 'refrence_summary','generated_summary',
            'ROUGE-1_Precision', 'ROUGE-1_Recall', 'ROUGE-1_F1',
            'ROUGE-2_Precision', 'ROUGE-2_Recall', 'ROUGE-2_F1',
            'ROUGE-L_Precision', 'ROUGE-L_Recall', 'ROUGE-L_F1',
        ]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        data = {
        'Case_ID': get_Case_ID(iteration),
        'refrence_summary': refrence_summary,
        'generated_summary': generated_summary,
        'ROUGE-1_Precision': round(score['rouge1'].precision, 4),
        'ROUGE-1_Recall': round(score['rouge1'].recall, 4),
        'ROUGE-1_F1': round(score['rouge1'].fmeasure, 4),
        'ROUGE-2_Precision': round(score['rouge2'].precision, 4),
        'ROUGE-2_Recall': round(score['rouge2'].recall, 4),
        'ROUGE-2_F1': round(score['rouge2'].fmeasure, 4),
        'ROUGE-L_Precision': round(score['rougeL'].precision, 4),
        'ROUGE-L_Recall': round(score['rougeL'].recall, 4),
        'ROUGE-L_F1': round(score['rougeL'].fmeasure, 4),
        }
        # print(data)  # Debugging line to check the output
        writer.writerow(data)



In [20]:
def run_process(start_iteration=0, end_iteration=250, iteration_file='iteration_tracker.txt'):
    try:
        # Check the last iteration completed from the file
        with open(iteration_file, 'r') as file:
            start_iteration = int(file.read().strip()) + 1
    except FileNotFoundError:
        # If the file does not exist, start from the provided or default start_iteration
        pass

    for iteration in range(start_iteration, end_iteration):
        print(f"Processing iteration {iteration}")
        process_instances(iteration)
        
        # Update the iteration file with the last completed iteration
        with open(iteration_file, 'w') as file:
            file.write(str(iteration))
        print(f"Completed iteration {iteration}, resuming will start at iteration {iteration + 1}")

In [ ]:
run_process()